In [1]:
import re
from collections import Counter
from collections import defaultdict
from copy import deepcopy
from tqdm.notebook import tqdm

In [2]:
pattern = r"(\d+),(\d+),(\d+)~(\d+),(\d+),(\d+)"

In [3]:
data = open("input/22").read().splitlines()

In [4]:
class Brick:
    def __init__(self, poses, name):
        self.x1 = poses[0]
        self.y1 = poses[1]
        self.z1 = poses[2]
        self.x2 = poses[3]
        self.y2 = poses[4]
        self.z2 = poses[5]
        
        self.blocks = set()
        self.min_z = None
        self.name = name        
        self.extract_blocks()
        
    def __repr__(self):
        return str(self.name) + str(self.blocks)
        
    def extract_blocks(self):
        self.min_z = min(self.z1, self.z2)
        self.blocks = set()
        for x in range(self.x1, self.x2 + 1):
            for y in range(self.y1, self.y2 + 1):
                for z in range(self.z1, self.z2 + 1):
                    self.blocks.add((x, y, z))
    
    def move_down(self):
        self.z1 -= 1
        self.z2 -= 1
        self.extract_blocks()
        return True
            

In [5]:
raw_bricks = []
for idx, line in enumerate(data):
    m = re.match(pattern, line)
    raw_bricks.append(Brick(list(map(int, m.groups())), idx))
    
brick_map = {}
for b in raw_bricks:
    brick_map[b.name] = b

In [6]:
def sort_bricks(bricks):
    z_values = []
    for brick in bricks:
        z_values.append([brick.min_z, brick.name, brick])
    z_values.sort()
    return [c for a, b, c in z_values]

In [7]:
bricks = sort_bricks(raw_bricks)

In [8]:
brick_order = []
for b in bricks:
    brick_order.append(b.name)

In [9]:
def drop_brick(brick):
    brick_name = brick.name
    candidates = brick_order[:brick_order.index(brick_name)]
    move_down = True
    blocked_by_candidates = set()
    for can in candidates:
        for block in brick_map[can].blocks:
            blocked_by_candidates.add((block[0], block[1], block[2] + 1))
    
    for pos in brick.blocks:
        if pos[2] == 1:
            return False
        
        if pos in blocked_by_candidates:
            return False
      
    if move_down:
        brick.move_down()
        return True
    return False

In [10]:
stable_set = set()
while True:
    stuff_happened = False
    for b in bricks:
        if b.name in stable_set:
            continue
        result = drop_brick(b)
        if result:
            stuff_happened = True
        else:
            stable_set.add(b.name)
        
    print(f"\rStable: {len(stable_set)} / {len(bricks)}", end="")
    if not stuff_happened:
        break

Stable: 1210 / 1210

## Some helper dictionaries

In [11]:
# Store all positions
all_poses = defaultdict(int)
for b in bricks:
    for pos in b.blocks:
        all_poses[pos] = b.name

In [12]:
# Calculate who supports every brick
supported_by = defaultdict(set)
for b in bricks:
    for pos in b.blocks:
        below = (pos[0], pos[1], pos[2] - 1)
        if below in all_poses:
            if all_poses[below] == b.name:
                continue
            supported_by[b.name].add(all_poses[below])

In [13]:
# Flipped of above
flipped = defaultdict(list)
for k, v in supported_by.items():
    for vv in v:
        flipped[vv].append(k)

In [14]:
def remove_x(brick):
    if brick.name not in flipped.keys():
        return True
    
    supports = flipped[brick.name]    
    found = set()
    for sup in supports:
        for key, val in flipped.items():
            if key == brick.name:
                continue
            if sup in val:
                found.add(sup)
    return len(supports) == len(found)

In [15]:
part1, part2 = [], []
for b in bricks:
    res = remove_x(b)
    part1.append(res)
    if not res:
        part2.append(b.name)

In [16]:
print("Answer #1:", sum(part1))

Answer #1: 393


# Part 2

In [17]:
def remove(brick_name):
    to_remove_list = [brick_name]
    removed = set()
    while True:
        if len(to_remove_list) == 0:
            break
        to_remove = to_remove_list.pop()
        removed.add(to_remove)
        cur = flipped[to_remove]
        for elem in cur:
            for brick in supported_by[elem]:
                if brick in removed:
                    continue
                elif brick in to_remove_list:
                    continue
                else:
                    break
            else:   
                to_remove_list.append(elem)
        
    return len(removed) - 1

In [18]:
res = []
for elem in part2:
    res.append(remove(elem))

In [19]:
sum(res)

58440